## XGBSurv Breslow Benchmark

In [1]:
from xgbsurv.datasets import (load_metabric, load_flchain, load_rgbsg, load_support, load_tcga)
from xgbsurv import XGBSurv
from xgbsurv.evaluation import cindex_censored, ibs
from xgbsurv.models.utils import sort_X_y
import os
import numpy as np
import pandas as pd
from scipy.stats import uniform as scuniform
from scipy.stats import randint as scrandint
from scipy.stats import loguniform as scloguniform 
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, LabelBinarizer, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.decomposition import PCA
# import models
from xgbsurv.models.breslow_final import breslow_likelihood,get_cumulative_hazard_function_breslow
from xgbsurv.models.efron_final import efron_likelihood
from xgbsurv.models.cind_final import cind_loss
from xgbsurv.models.deephit_pycox_final import deephit_loss1_pycox
from xgbsurv.models.eh_aft_final import aft_likelihood
from xgbsurv.models.eh_ah_final import ah_likelihood
from pycox.evaluation import EvalSurv
from sklearn.utils.fixes import loguniform
from xgbsurv.models.utils import sort_X_y_pandas, transform_back, transform
from xgbsurv.preprocessing.dataset_preprocessing import discretizer_df
import os
import sys


In [2]:
current_path = os.getcwd() 
one_level_up = os.path.abspath(os.path.join(current_path,  ".."))
two_levels_up = os.path.abspath(os.path.join(current_path,  "..",".."))
one_level_up
sys.path.append(one_level_up+'/gbdt_pipeline')
from gbdt_pipeline import get_gbdt_pipeline, train_gbdt

current path /Users/JUSC/Documents/xgbsurv/experiments/efron


## Set Parameters

In [3]:
# set parameters
n_outer_splits = 5
n_inner_splits = 5
rand_state = 42
n_iter = 50 
early_stopping_rounds=10
base_score = 0.0
validation_size = 0.2



# set seed for scipy
np.random.seed(rand_state)


## Custom Splitting

In [4]:
class CustomSplit(StratifiedKFold):
    def __init__(self, n_splits=5, shuffle=True, random_state=None):
        super().__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    def split(self, X, y, groups=None):
        try:
            if y.shape[1]>1:
                y = y[:,0]
        except:
            pass
        bins = np.sign(y)
        return super().split(X, bins, groups=groups)

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

outer_custom_cv = CustomSplit(n_splits=n_outer_splits, shuffle=True, random_state=rand_state)


## Set Model & Train Test Evaluate

In [5]:
data_set_fns = [load_metabric,  load_flchain, load_rgbsg, load_support] 
metrics_list = []

for idx, dataset in enumerate(data_set_fns):
    model = 'efron'
    # get name of current dataset
    data = dataset(path="/Users/JUSC/Documents/xgbsurv/xgbsurv/datasets/data/", as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        pi = get_gbdt_pipeline()
        metric = train_gbdt(X, y, i, pi, train_index, test_index, model, dataset_name, validation_size)
        print(metric)
        metrics_list.append(metric)



METABRIC_adapted.csv
0
estimator XGBSurv(base_score=0.0, booster=None, callbacks=None, colsample_bylevel=None,
        colsample_bynode=None, colsample_bytree=None,
        disable_default_eval_metric=True, early_stopping_rounds=10,
        enable_categorical=False,
        eval_metric=CPUDispatcher(<function breslow_likelihood at 0x29945fce0>),
        feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
        importance_type=None, interaction_constraints=None, learning_rate=None,
        max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
        max_delta_step=None, max_depth=None, max_leaves=None,
        min_child_weight=None, missing=nan, monotone_constraints=None,
        n_estimators=100, n_jobs=None, num_parallel_tree=None,
        objective=CPUDispatcher(<function breslow_objective at 0x299471300>), ...)
estimator XGBSurv(base_score=0.0, booster=None, callbacks=None, colsample_bylevel=None,
        colsample_bynode=None, colsample_bytree=None,
        di

ZeroDivisionError: division by zero

## TCGA Train, Test, Evaluation

In [ ]:
cancer_types = [
    'BLCA',
    'BRCA',
    'HNSC',
    'KIRC',
    'LGG',
    'LIHC',
    'LUAD',
    'LUSC',
    'OV',
    'STAD'
    ]

metrics_list = []

for i, cancer_type in enumerate(cancer_types):
    model = 'efron'
    # get name of current dataset
    data = load_tcga(path="/Users/JUSC/Documents/xgbsurv/xgbsurv/datasets/data/",cancer_type=cancer_type, as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        pi = get_gbdt_pipeline()
        metric = train_gbdt(X, y, i, pi, train_index, test_index, model, dataset_name, validation_size,tcga=True)
        print(metric)
        metrics_list.append(metric)

BLCA_adapted.csv
0
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Concordance Index 0.2292817679558011
Integrated Brier Score: 0.37789605765175127
{'model': 'breslow', 'dataset': 'BLCA', 'cindex_train': [0.5604557690180316], 'cindex_test': [0.2292817679558011], 'ibs_train': [0.21391197072804385], 'ibs_score_test': [0.37789605765175127]}
1
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Concordance Index 0.3872200983069361
Integrated Brier Score: 0.29957421380493215
{'model': 'breslow', 'dataset': 'BLCA', 'cindex_train': [0.5779632721202004], 'cindex_test': [0.3872200983069361], 'ibs_train': [0.20834039825673512], 'ibs_score_test': [0.29957421380493215]}
2
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Concordance Index 0.3924731182795699
Integrated Brier Score: 0.3080347659663883
{'model': 'breslow', 'dataset': 'BLCA', 'cindex_train': [0.5912974225311324], 'cindex_test': [0.3924731182795699], 'ibs_train': [0.21432256207037484], 'ibs_score_test':

In [ ]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df

,model,dataset,cindex_train,cindex_test,ibs_train,ibs_score_test
0,breslow,BLCA,0.560456,0.229282,0.213912,0.377896
1,breslow,BLCA,0.577963,0.387220,0.208340,0.299574
2,breslow,BLCA,0.591297,0.392473,0.214323,0.308035
3,breslow,BLCA,0.598222,0.371134,0.208545,0.327388
4,breslow,BLCA,0.596315,0.342599,0.191378,0.237748
5,breslow,BRCA,0.448877,0.429253,0.154505,0.188507
6,breslow,BRCA,0.802661,0.486343,0.183438,0.268922
7,breslow,BRCA,0.623802,0.357836,0.154165,0.192019
8,breslow,BRCA,0.665676,0.474730,0.154030,0.168100
9,breslow,BRCA,0.565007,0.357171,0.154695,0.233703


,dataset,ibs_train_mean,ibs_train_std,ibs_test_mean,ibs_test_std
0,BLCA,0.2142,0.0043,0.2228,0.0060
0,BRCA,0.1905,0.0046,0.1964,0.0092
0,HNSC,0.1833,0.0055,0.2038,0.0074
